### Script de extração de dados do arquivo TXT

Este script é utilizado para extrair dados de um arquivo result.txt, que contém informações sobre discos, incluindo os artistas, nomes dos discos, caminhos das imagens (capa e verso), ano de lançamento, gênero musical e nacionalidade. O script processa essas informações e as insere em uma tabela no banco de dados PostgreSQL.

A celula abaixo importa as bibliotecas usadas e puxa as variaveis de conexão do banco do arquivo .env 

In [47]:
import psycopg2
import os
from dotenv import load_dotenv

load_dotenv()

dbname = os.getenv('DB_NAME')
user = os.getenv('DB_USER')
password = os.getenv('DB_PASSWORD')
host = os.getenv('DB_HOST')
port = os.getenv('DB_PORT')

Esssa celula se conecta a um banco de dados PostgreSQL e lê dados de um arquivo chamado result.txt. Primeiro, ele estabelece a conexão e cria um cursor para executar comandos. Em seguida, o arquivo é aberto e suas linhas são lidas. Para cada linha, o script divide os dados em partes, extraindo o nome do artista, a nacionalidade e o gênero. Ele verifica se o artista já existe na tabela artistas. Se não existir, insere os dados do artista no banco de dados. Após inserir os dados, as alterações são confirmadas. O script então busca todos os registros na tabela artistas e imprime cada um deles no console. Se ocorrer algum erro durante o processo, uma mensagem de erro é exibida. Por fim, o cursor e a conexão com o banco de dados são fechados para garantir que os recursos sejam liberados corretamente.

In [48]:
try:

    conn = psycopg2.connect(
        dbname=dbname,
        user=user,
        password=password,
        host=host,
        port=port
    )
    cur = conn.cursor()

    with open('txt/result/result.txt') as data_txt:
        datas = data_txt.readlines()

    for data in datas:
        data = data.split(',')
        nome = data[0].strip()
        nacionalidade = data[-1].strip()
        genero = data[-2].strip()

        cur.execute("SELECT * FROM artistas WHERE nome = %s", (nome,))
        if not cur.fetchone():  # Se não encontrar o artista, insere na tabela
            cur.execute(
                "INSERT INTO artistas (nome, nacionalidade, genero) VALUES (%s, %s, %s)",
                (nome, nacionalidade, genero)
            )

    conn.commit()


    # Verificando os dados inseridos
    cur.execute("SELECT * FROM artistas;")
    resultados = cur.fetchall()

    # Exibindo os resultados
    for row in resultados:
        print(row)


except Exception as e:
    print(f"Ocorreu um erro: {e}")

finally:
    if cur:
        cur.close()
    if conn:
        conn.close()

(1, "Guns n' roses", 'EUA', 'Hard Rock')
(2, 'SP Metal Vol. 2', 'Brasil', 'Heavy Metal / Rock')
(3, 'A-ha', 'Noruega', 'Pop / Synthpop')
(4, 'David Bowie', 'Reino Unido', 'Rock / Pop')
(5, 'Nazareth', 'Escócia', 'Rock')
(6, 'Venom', 'Reino Unido', 'Black Metal')
(7, 'Judas Priest', 'Reino Unido', 'Heavy Metal')
(8, 'Saxon', 'Reino Unido', 'Heavy Metal')
(9, 'Van halen', 'EUA', 'Hard Rock')
(10, 'Running wild', 'Alemanha', 'Heavy Metal')
(11, 'Accept', 'Alemanha', 'Heavy Metal')
(12, 'The alan parsons project', 'Reino Unido', 'Progressive Rock')
(13, "Bagga's guru", 'Brasil', 'Rock')
(14, 'Abba', 'Suécia', 'Pop')
(15, '12 super tops', 'Brasil', 'Compilação / Vários Gêneros')
(16, 'Spin doctors', 'EUA', 'Rock / Alternative Rock')
(17, 'Bread', 'EUA', 'Soft Rock')
(18, 'The rolling stones', 'Reino Unido', 'Rock')
(19, 'Bob marley and The wailers', 'Jamaica', 'Reggae')
(20, 'George michael and queen with lisa stanfield', 'Reino Unido', 'Pop / Rock')
(21, 'Rainbow', 'Reino Unido', 'Hard Roc

Este script se conecta a um banco de dados PostgreSQL e lê dados de um arquivo chamado result.txt. Após estabelecer a conexão e criar um cursor, o arquivo é aberto e suas linhas são lidas. Para cada linha, o script processa as informações do disco, extraindo o nome do artista, o nome do disco, os caminhos das imagens, o ano de lançamento, o gênero e a nacionalidade. A data de lançamento é formatada como YYYY-01-01. O script verifica se o artista existe na tabela artistas e, se existir, obtém seu artista_id. Em seguida, ele verifica se o disco já está na tabela discos para evitar duplicatas. Se o disco não estiver presente, ele insere os dados do disco na tabela. Após a inserção, as alterações são confirmadas. O script então busca todos os registros da tabela discos e imprime cada um deles no console. Caso ocorra um erro durante o processo, uma mensagem de erro é exibida. Por fim, o cursor e a conexão com o banco de dados são fechados para liberar os recursos utilizados.

In [49]:
try:
    conn = psycopg2.connect(
        dbname=dbname,
        user=user,
        password=password,
        host=host,
        port=port
    )
    cur = conn.cursor()

    with open('txt/result/result.txt') as data_txt:
        datas = data_txt.readlines()

    for data in datas:
        data = data.strip().split(',')
        artista_nome = data[0].strip()
        nome = data[1].strip()
        capa_caminho = data[2].strip()
        verso_caminho = data[3].strip()
        ano = data[4].strip()
        genero = data[5].strip()
        nacionalidade = data[6].strip()
        data_lancamento = f"{ano}-01-01"

        cur.execute("SELECT artista_id FROM artistas WHERE nome = %s", (artista_nome,))
        artista = cur.fetchone()

        if artista:
            artista_id = artista[0]
            cur.execute("SELECT 1 FROM discos WHERE nome = %s AND artista_id = %s", (nome, artista_id))
            if cur.fetchone() is None:
                cur.execute(
                    "INSERT INTO discos (nome, artista_id, data_lancamento, capa_caminho, verso_caminho) VALUES (%s, %s, %s, %s, %s)",
                    (nome, artista_id, data_lancamento, capa_caminho, verso_caminho)
                )

    conn.commit()

    cur.execute("SELECT * FROM discos")
    discos_inseridos = cur.fetchall()

    for disco in discos_inseridos:
        print(disco)

except Exception as e:
    print(f"Ocorreu um erro: {e}")

finally:
    if cur:
        cur.close()
    if conn:
        conn.close()

(1, 'Live ?!*@ Like a Suicide', 1, datetime.date(1986, 1, 1), 'IMG-20241028-WA0020.jpg', 'IMG-20241028-WA0021.jpg')
(2, 'SP Metal Vol. 2', 2, datetime.date(1988, 1, 1), 'IMG-20241028-WA0022.jpg', 'IMG-20241028-WA0023.jpg')
(3, 'Burning high and low', 3, datetime.date(1999, 1, 1), 'IMG-20241028-WA0024.jpg', 'IMG-20241028-WA0025.jpg')
(4, 'Labyrinth', 4, datetime.date(1986, 1, 1), 'IMG-20241028-WA0030.jpg', 'IMG-20241028-WA0031.jpg')
(5, 'Bazamanaz', 5, datetime.date(1973, 1, 1), 'IMG-20241028-WA0033.jpg', 'IMG-20241028-WA0035.jpg')
(6, 'Welcome to hell', 6, datetime.date(1981, 1, 1), 'IMG-20241028-WA0037.jpg', 'IMG-20241028-WA0039.jpg')
(7, 'Unleashed in the east', 7, datetime.date(1979, 1, 1), 'IMG-20241028-WA0041.jpg', 'IMG-20241028-WA0043.jpg')
(8, 'Crusader', 8, datetime.date(1984, 1, 1), 'IMG-20241028-WA0045.jpg', 'IMG-20241028-WA0049.jpg')
(9, '1984', 9, datetime.date(1984, 1, 1), 'IMG-20241028-WA0050.jpg', 'IMG-20241028-WA0056.jpg')
(10, 'Gates to purgatory', 10, datetime.date(19